## Requirements
```
pip install transformers
pip install datasets
pip install nltk
```
Installeer [CUDA](https://developer.nvidia.com/cuda-downloads) en [PyTorch](https://pytorch.org/get-started/locally/) als je op GPU wilt trainen anders installeer alleen PyTorch op CPU

## Source
[source](https://colab.research.google.com/drive/1z-Zl2hftMrFXabYfmz8o9YZpgYx6sGeW?usp=sharing#scrollTo=OG82dfGbK63x)

# Load and install

In [1]:
import transformers
import datasets
import torch
import logging

c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Preprocessing
sep_token = '<sep>'
dataset_name = "wiselinjayajos/squad_modified_for_t5_qg"
models_dir = "saved_models/t5_base-squad_mod"
checkpoint = 't5-base'
max_input_length = 512
max_target_length = 64

## Training
learning_rate = 1e-4
num_epochs = 7

In [3]:
dataset = datasets.load_dataset(dataset_name)

Found cached dataset parquet (C:/Users/manuv/.cache/huggingface/datasets/wiselinjayajos___parquet/wiselinjayajos--squad_modified_for_t5_qg-a090cde2e8fceb0a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 2/2 [00:00<00:00, 20.30it/s]


In [4]:
dataset["train"][0]

{'context': 'generate questions: Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'questions': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? {sep_token} What is in front of the Notre Dame Main Building? {sep_token} The Basilica of the Sacred heart at Notre Dame is beside to which structure? {sep_token} What is the Grotto

In [5]:
model = transformers.T5ForConditionalGeneration.from_pretrained(checkpoint)
tokenizer = transformers.T5TokenizerFast.from_pretrained(checkpoint)

c:\Users\manuv\miniconda3\envs\t5_qg\Lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [6]:
tokenizer.sep_token = sep_token
tokenizer.add_tokens([sep_token])
model.resize_token_embeddings(len(tokenizer))

Embedding(32102, 768)

In [11]:
# Tokenize examples
def convert_to_features(example_batch):

    input_encodings = tokenizer.batch_encode_plus(example_batch['context'], 
                                                  max_length=max_input_length, 
                                                  add_special_tokens=True,
                                                  truncation=True, 
                                                  pad_to_max_length=True)
    
    target_encodings = tokenizer.batch_encode_plus(example_batch['questions'], 
                                                   max_length=max_target_length, 
                                                   add_special_tokens=True,
                                                   truncation=True, pad_to_max_length=True)
                                                   
    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'decoder_input_ids': target_encodings['input_ids'],
        'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings

def add_eos_examples(example):
    example['context'] = example['context'] + " " + sep_token
    example['questions'] = example['questions'] + " " + sep_token
    return example


def add_special_tokens(example):
  example['questions'] = example['questions'].replace("{sep_token}", sep_token)
  return example

In [12]:
tokenized_dataset  = dataset.map(add_eos_examples)
tokenized_dataset = tokenized_dataset.map(add_special_tokens)
tokenized_dataset  = tokenized_dataset.map(convert_to_features,  batched=True)

Loading cached processed dataset at C:\Users\manuv\.cache\huggingface\datasets\wiselinjayajos___parquet\wiselinjayajos--squad_modified_for_t5_qg-a090cde2e8fceb0a\0.0.0\2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec\cache-86b2ba036c9ec29b.arrow
Loading cached processed dataset at C:\Users\manuv\.cache\huggingface\datasets\wiselinjayajos___parquet\wiselinjayajos--squad_modified_for_t5_qg-a090cde2e8fceb0a\0.0.0\2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec\cache-aeff52f71385625f.arrow
Loading cached processed dataset at C:\Users\manuv\.cache\huggingface\datasets\wiselinjayajos___parquet\wiselinjayajos--squad_modified_for_t5_qg-a090cde2e8fceb0a\0.0.0\2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec\cache-65b56a4d05dede59.arrow
Loading cached processed dataset at C:\Users\manuv\.cache\huggingface\datasets\wiselinjayajos___parquet\wiselinjayajos--squad_modified_for_t5_qg-a090cde2e8fceb0a\0.0.0\2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934

<class 'list'> 1000
<class 'list'> 1000


Map:   5%|▌         | 1000/18896 [00:00<00:05, 3256.59 examples/s]

<class 'list'> 1000
<class 'list'> 1000


Map:  11%|█         | 2000/18896 [00:00<00:04, 3640.21 examples/s]

...

Map:   0%|          | 0/2067 [00:00<?, ? examples/s]               

<class 'list'> 1000
<class 'list'> 1000


Map:  48%|████▊     | 1000/2067 [00:00<00:00, 3890.17 examples/s]

<class 'list'> 1000
<class 'list'> 1000


<class 'list'> 67
<class 'list'> 67


In [9]:
tokenized_dataset["train"][0]["questions"]

'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? <sep> What is in front of the Notre Dame Main Building? <sep> The Basilica of the Sacred heart at Notre Dame is beside to which structure? <sep> What is the Grotto at Notre Dame? <sep> What sits on top of the Main Building at Notre Dame? <sep> <sep>'

In [10]:
tokenized_dataset = tokenized_dataset.remove_columns(
    ["context", "questions"]
)

train_dataset = tokenized_dataset["train"]
valid_dataset = tokenized_dataset["validation"]

columns = ['input_ids', 'decoder_input_ids', 'attention_mask', 'decoder_attention_mask']
train_dataset.set_format(type='torch', columns=columns)
valid_dataset.set_format(type='torch', columns=columns)

In [11]:
torch.save(train_dataset, 'train_data.pt')
torch.save(valid_dataset, 'valid_data.pt')

In [12]:
from typing import Dict, List

class T2TDataCollator():
    def __call__(self, batch: List) -> Dict[str, torch.Tensor]:
        """
        Take a list of samples from a Dataset and collate them into a batch.
        Returns:
        A dictionary of tensors
        """

        input_ids = torch.stack([example['input_ids'] for example in batch])
        lm_labels = torch.stack([example['decoder_input_ids'] for example in batch])
        lm_labels[lm_labels[:, :] == 0] = -100 
        attention_mask = torch.stack([example['attention_mask'] for example in batch])
        decoder_attention_mask = torch.stack([example['decoder_attention_mask'] for example in batch])

        return {
            'input_ids': input_ids, 
            'attention_mask': attention_mask,
            'labels': lm_labels, 
            'decoder_attention_mask': decoder_attention_mask
        }

In [13]:
training_args = transformers.TrainingArguments(output_dir=models_dir, 
                                  per_device_train_batch_size=4, 
                                  per_device_eval_batch_size=4,
                                  gradient_accumulation_steps=16,
                                  learning_rate=learning_rate, 
                                  num_train_epochs=num_epochs,
                                  logging_steps=100,
                                  run_name="t5_qg_squad",
                                  evaluation_strategy="steps",
                                  save_steps=500)

In [14]:
logger = logging.getLogger(__name__)

# Initialize our Trainer
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=T2TDataCollator()
)


In [15]:
if model.device.type == 'cuda':
    print('Model is on GPU')
else:
    print('Model is on CPU')

Model is on GPU


In [16]:
# Training https://colab.research.google.com/drive/1z-Zl2hftMrFXabYfmz8o9YZpgYx6sGeW?usp=sharing#scrollTo=OG82dfGbK63x
trainer.train()#3u45-4u op RTX 3060 12GB 

c:\Users\manuv\miniconda3\envs\t5_qg\Lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  5%|▍         | 100/2065 [09:52<3:13:04,  5.90s/it]

{'loss': 2.6627, 'learning_rate': 9.515738498789348e-05, 'epoch': 0.34}


                                                    
  5%|▍         | 100/2065 [10:54<3:13:04,  5.90s/it]

{'eval_loss': 1.982875943183899, 'eval_runtime': 62.9394, 'eval_samples_per_second': 32.841, 'eval_steps_per_second': 8.214, 'epoch': 0.34}


 10%|▉         | 200/2065 [20:47<3:03:30,  5.90s/it] 

...

                                                     
 97%|█████████▋| 2000/2065 [3:34:24<06:11,  5.72s/it]

{'eval_loss': 1.5853705406188965, 'eval_runtime': 61.133, 'eval_samples_per_second': 33.812, 'eval_steps_per_second': 8.457, 'epoch': 6.77}


100%|██████████| 2065/2065 [3:40:42<00:00,  6.41s/it]

{'train_runtime': 13242.2891, 'train_samples_per_second': 9.989, 'train_steps_per_second': 0.156, 'train_loss': 1.6986265316425166, 'epoch': 6.99}


TrainOutput(global_step=2065, training_loss=1.6986265316425166, metrics={'train_runtime': 13242.2891, 'train_samples_per_second': 9.989, 'train_steps_per_second': 0.156, 'train_loss': 1.6986265316425166, 'epoch': 6.99})

In [17]:
# save the model
trainer.save_model(models_dir)

In [9]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast
# load the saved model
loaded_model = T5ForConditionalGeneration.from_pretrained('saved_models/t5_base-squad_mod')

In [12]:
models_dir = "saved_models/t5_base-squad_mod"
checkpoint = 't5-base'
tokenizer = T5TokenizerFast.from_pretrained(checkpoint)

c:\Users\manuv\miniconda3\envs\t5_qg\Lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [13]:
def run_model(input_string, **generator_args):
    generator_args = {
    "max_length": 512,
    "num_beams": 4,# bij grotere num_beams is trager maar complexere vragen(niet per se betere vragen)
    "length_penalty": 1.5,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
    }
    input_string = "generate questions: " + input_string + " </s>"
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = loaded_model.generate(input_ids, **generator_args)
    output = tokenizer.batch_decode(res, skip_special_tokens=True)
    output = [item.split("<sep>") for item in output]
    return output

In [14]:
context = """
Cheese is an ancient food whose origins predate recorded history. There is no conclusive evidence indicating where cheesemaking originated, whether in Europe, Central Asia or the Middle East. Earliest proposed dates for the origin of cheesemaking range from around 8000 BCE, when sheep were first domesticated. Since animal skins and inflated internal organs have, since ancient times, provided storage vessels for a range of foodstuffs, it is probable that the process of cheese making was discovered accidentally by storing milk in a container made from the stomach of an animal, resulting in the milk being turned to curd and whey by the rennet from the stomach.[7] There is a legend—with variations—about the discovery of cheese by an Arab trader who used this method of storing milk.[8]

The earliest evidence of cheesemaking in the archaeological record dates back to 5500 BCE and is found in what is now Kuyavia, Poland, where strainers coated with milk-fat molecules have been found.[9]

Cheesemaking may have begun independently of this by the pressing and salting of curdled milk to preserve it. Observation that the effect of making cheese in an animal stomach gave more solid and better-textured curds may have led to the deliberate addition of rennet. Early archeological evidence of Egyptian cheese has been found in Egyptian tomb murals, dating to about 2000 BCE.[10] A 2018 scientific paper stated that the world's oldest cheese, dating to approximately 1200 BCE (3200 years before present), was found in ancient Egyptian tombs.[11][12]

The earliest cheeses were likely quite sour and salty, similar in texture to rustic cottage cheese or feta, a crumbly, flavorful Greek cheese. Cheese produced in Europe, where climates are cooler than the Middle East, required less salt for preservation. With less salt and acidity, the cheese became a suitable environment for useful microbes and molds, giving aged cheeses their respective flavors. The earliest ever discovered preserved cheese was found in the Taklamakan Desert in Xinjiang, China, dating back as early as 1615 BCE (3600 years before present).
"""

run_model(context)

[["What is an ancient food whose origins predate recorded history? When were sheep first domesticated? Where is the earliest evidence of cheesemaking found? In what year was the world's oldest cheese discovered?"]]

## TODO

* Evaluatie van model toevoegen(BLEU,ROUGE,METEOR,QA model)
* Inference op GF
* Op andere dataset trainen
* Msschn andere params gebruiken